# NASA AirFoil Self-Noise Dataset

### Introducción

Elijo este dataset para el TP de AM1 acorde a que es mi área de estudio de carrera de grado. Yo soy Ingeniero Industrial Mecánico especializado en Fluidos y Energía, y el estudio del perfil aerodinámico fue central en el desarrollo de la carrera. Al iniciar mi especialización en Inteligencia Artificial, uno de mis objetivos era integrar ambos campos de conocimiento y enriquecerlos mutuamente.

De esta manera, la idea es que el análisis de datos sea complementado con análisis teórico del experimento.

## Dataset

Conjunto de datos de la NASA obtenido de una serie de pruebas aerodinámicas y acústicas de secciones de alas bidimensionales y tridimensionales realizadas en un túnel de viento anecoico. El conjunto de datos de la NASA comprende perfiles aerodinámicos de diferentes tamaños a varias velocidades de túnel de viento y ángulos de ataque. Se busca predecir el valor del nivel de presión sonora, en decibeles (SSPL).

Link al dataset: [https://www.kaggle.com/datasets/fedesoriano/airfoil-selfnoise-dataset/](https://www.kaggle.com/datasets/fedesoriano/airfoil-selfnoise-dataset/)

Este dataset comprende un ensayo de laboratorio de 1989 ([Referencia](https://ntrs.nasa.gov/api/citations/19890016302/downloads/19890016302.pdf)) con el objetivo de encontrar relaciones empirícas para determinar el ruido provocado por la propia interacción de un perfil aerodinámico NACA 0012 y el campo de velocidades.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay


In [6]:
#Cargar el dataset
NASA_df = pd.read_csv("./datasets/AirfoilSelfNoise.csv")
NASA_df.describe()

,f,alpha,c,U_infinity,delta,SSPL
count,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000
mean,2886.380572,6.782302,0.136548,50.860745,0.011140,124.835943
std,3152.573137,5.918128,0.093541,15.572784,0.013150,6.898657
min,200.000000,0.000000,0.025400,31.700000,0.000401,103.380000
25%,800.000000,2.000000,0.050800,39.600000,0.002535,120.191000
50%,1600.000000,5.400000,0.101600,39.600000,0.004957,125.721000
75%,4000.000000,9.900000,0.228600,71.300000,0.015576,129.995500
max,20000.000000,22.200000,0.304800,71.300000,0.058411,140.987000


In [7]:
NASA_df.head()

,f,alpha,c,U_infinity,delta,SSPL
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


Este dataset se compone de 5 features:

* Frecuencia (f)[Hz]
* Ángulo de ataque ($\alpha$)[°]
* Largo de cuerda (c)[m]
* Velocidad en el infinito ($U_{inf}$)[m/s]
* Espesor del desplazamiento de la capa limite en la succión ($\delta_s$)[m]

Y el target es:

* Ruido autoproducido (SSPL)[db]

### Entendiendo el experimento

Acá juegan 3 variables directas relacionadas con el ensayo, $U_{inf}$ y $\alpha$ caracterizan al viento y $c$ caracteriza al perfil.

Como se ve en la imagen, la cuerda del perfil es la distancia entre el borde de ataque (leading edge) y el borde de salida (trailing edge), para el caso del perfil de estudio [NACA0012](http://airfoiltools.com/airfoil/details?airfoil=n0012-il) es un perfil simétrico por lo que se puede entender como el largo del perfil.

![AltText](./images/Sustentación.jpg)

El $U_{inf}$ es la velocidad del viento relativo al perfil previa a la interacción con el mismo y $\alpha$ es la diferencia angular entre los vectores de proyección de la cuerda y el vector del campo de velocidades. Como el perfil de estudio es simétrico no tiene relevancia estudiar casos con ángulo de ataque negativo y por eso el mínimo $\alpha$ es 0.

El concepto básico de la sustentación sucede como una fuerza resultante de un diferencial de presiones (más presión debajo del perfil). Este $\nabla_p$ existe debido a que por el principio de Bernoulli la $v^2 + \nabla_p$ debe ser constante, al generar una disminución del área de pasaje por encima del perfil y por conservación de la masa, el caudal tiene que mantenerse, debe aumentar la velocidad, lo que hace que la presión disminuya.

Esta relación entre velocidad y presión, va variando con el ángulo de ataque generando diferentes resultantes de sustentación, llegando a un máximo al cual se pierde totalmente la sustentación.

![AltText](./images/angulodeataque.jpg)

Lo que para este perfil particular se observa en la relación entre $c_L$ (coeficiente de sustentación) vs $\alpha$. (Las diferentes curvas son para valores de velocidad).

![AltText](./images/Coeficiente_Sustentacion.jpg)

En la última imagen se pueden observar 5 tipos de ruido que se pueden generar con combinaciones de estas ultimas 2 variables y que se deben a diferentes fenómenos de la aerodinámica. En particular, si fuese a nivel laboral (o sea más tiempo) un gran candidato como modelo base sería poder encontrar una relación teórica del gradiente de presiones y predecir cómo interactúa con cada tipo de ruido. Esto permitiría poder comparar un modelo físico (con muchas suposiciones) vs el modelo de Aprendizaje Automático.

![AltText](./images/NoiseTypes.jpg)



Restan dos features para estudiar:

Para el espesor de desplazamiento de capa límite no me voy a explayar mucho pues puede resultar muy complejo y no viene al caso de este TP, pero para entenderlo voy a resumir el concepto de capa límite.

El aire es un fluido, y como fluido la interacción con un objeto sólido no puede permitir que la velocidad relativa del aire no sea cero (pensemos en la partícula de aire que esta *pegada* al objeto). De esta manera se genera un perfil de velocidad que va variando hasta llegar nuevamente a la velocidad del infinito, en ese espacio el aire se comporta como un fluido viscoso y las partículas que están más cerca del perfil generan un rozamiento que va disminuyendo a medida que se aleja del objeto.

El $\delta_s$ es una manera de calcular el espesor en el que el aire se comporta como fluido.

![AltText](./images/DisplacementThickness.jpg)

Lo más importante acá es destacar que no es una magnitud medible, si no que se calcula, en particular para este caso (pág 9-14 de [NACA0012](http://airfoiltools.com/airfoil/details?airfoil=n0012-il)):

\begin{equation}
\delta_s = \delta_s(Re,c,\alpha)
\end{equation}

Siendo Re el Reynolds, un número adimensionado que depende de la velocidad $U_{inf}$ y propiedades del aire. 

Para lo que interesa de esta materia, este valor delta entonces no aporta valor nuevo si no que es función del resto de elementos. Habría que poder observar esto en el análisis de datos.

Por último la frecuencia, es quizás lo más sencillo, el ruido depende de la frecuencia a la que se mida y nos puede interesar los valores de un rango audible humano (20Hz - 20KHz) o por ejemplo en un caso de un aerogenerador, podría interesar medir el ruido en el rango audible de una especie de ave que lo pueda dañar.

![AltText](./images/champ_auditif_es.jpg)

En conclusión, el experimento mide ruido en diferentes frecuencias, usando valores de $\alpha$ $U_{inf}$ y c. Los valores de $\delta$ no aportarían información extra en un principio. A no ser que haya claro indicio de una mala medición, no deberíamos quitar ningún outlier puesto de que los resultados se deben a un fenómeno físico.